In [13]:
print("Content for requirements.txt")
#!pip install xgboost
#!pip install -U google-cloud-storage
#!pip uninstall -y google-cloud 
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn
#!pip install sklearn

Content for requirements.txt


In [64]:
from google.cloud import storage

import pandas as pd 
import numpy as np

In [66]:

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
# Classification metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import cohen_kappa_score
# Ignore ConvergenceWarning messages
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

In [138]:
# Online
#evaluate = pd.read_csv('gs://credit-risk-bucket/application_evaluate.csv', sep=",")
# local
evaluate = pd.read_csv('/home/humberto/vs-workspace/prj-3760-credit-assessment/3760creditrisk/model/record_eval1.csv', sep=",")
evaluate



,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [139]:
# Data Cleaning for a single record
evaluate['OCCUPATION_TYPE'].value_counts() 
evaluate['OCCUPATION_TYPE'].unique()
evaluate['OCCUPATION_TYPE'].replace([ 'Cleaning staff', 'Cooking staff', 'Accountants', 'Medicine staff'
        'Private service staff', 'Security staff','Waiters/barmen staff',
        'Low-skill Laborers', 'Realty agents', 'Secretaries', 'IT staff', 'HR staff'], 'Others', inplace= True)
evaluate['ORGANIZATION_TYPE'].value_counts()
evaluate['ORGANIZATION_TYPE'].replace(['School', 'Religion',
       'Other','Electricity', 'Medicine',
       'Transport: type 2',
       'Construction', 'Housing', 'Kindergarten', 'Trade: type 7',
       'Industry: type 11', 'Military', 'Services', 'Security Ministries',
       'Transport: type 4', 'Industry: type 1', 'Emergency', 'Security',
       'Trade: type 2', 'University', 'Transport: type 3', 'Police',
       'Business Entity Type 1', 'Postal', 'Industry: type 4',
       'Agriculture', 'Restaurant', 'Culture', 'Hotel',
       'Industry: type 7', 'Trade: type 3', 'Industry: type 3', 'Bank',
       'Industry: type 9', 'Insurance', 'Trade: type 6',
       'Industry: type 2', 'Transport: type 1', 'Industry: type 12',
       'Mobile', 'Trade: type 1', 'Industry: type 5', 'Industry: type 10',
       'Legal Services', 'Advertising', 'Trade: type 5', 'Cleaning',
       'Industry: type 13', 'Trade: type 4', 'Telecom',
       'Industry: type 8', 'Realtor', 'Industry: type 6'], 'Others', inplace = True)
outlier = evaluate[evaluate['DAYS_EMPLOYED'] == 365243]
evaluate['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True) # replace outlier by 
evaluate['DAYS_EMPLOYED'].fillna(np.mean(evaluate['DAYS_EMPLOYED']), inplace= True) # fill na with by mean of the column


In [140]:
# Feature engineering
evaluate['Credit_income_ratio'] = evaluate['AMT_CREDIT']/evaluate['AMT_INCOME_TOTAL']
evaluate['Anuity_income_ratio'] = evaluate['AMT_ANNUITY']/evaluate['AMT_INCOME_TOTAL'] 
evaluate['Credit_term'] = evaluate['AMT_ANNUITY']/evaluate['AMT_CREDIT'] #length of the payment in months since the annuity is the monthly amount due
evaluate['Days_employed_age'] = evaluate['DAYS_EMPLOYED']/evaluate['DAYS_BIRTH']

In [141]:

#load modelled columns from model (173 columns sample)
import pickle
with open('train_columns_pv1.pkl', 'rb') as f:
    train_ref_col = pickle.load(f)



In [142]:
# Convert categorical feature to numberic using one-hot encoding
evaluate.loc[0,'SK_ID_CURR'] = 100001
evaluate = pd.get_dummies(evaluate, drop_first= True)
evaluate

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Credit_income_ratio,Anuity_income_ratio,Credit_term,Days_employed_age
0,100001,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0.0,0.0,0.0,0.0,0.0,1.0,2.007889,0.121978,0.060749,0.067329


In [143]:
train_ref_col

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,0,0,0,1,0,0


In [145]:
aligned_eval, a2 = evaluate.align(train_ref_col, join='right', axis=1)
aligned_eval # a1 adjusted imported pickle






,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,100001,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
#Drop target if ir exists
aligned_eval = aligned_eval.drop('TARGET', axis=1)
aligned_eval


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,100001,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
# Verifications
"""
- CODE_GENDER_M
- CODE_GENDER_XNA
- EMERGENCYSTATE_MODE_Yes
- FLAG_OWN_CAR_Y
- FLAG_OWN_REALTY_Y
"""
aligned_eval['CODE_GENDER_M']

0   NaN
Name: CODE_GENDER_M, dtype: float64

In [156]:
import pickle
with open('credit_model_pv1.pkl', 'rb') as f:
    model = pickle.load(f)
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=0.7, enable_categorical=False,
                               gamma=0.3, gpu_id=-1, importance_type=None,
                               interaction_constraints='', learning_rate=0.05,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=3, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=4, num_parallel_tree=1, predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=1, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))

In [161]:

pred_class = model.predict(aligned_eval)
pred_class[0]

0

In [162]:
# Class Probabilities
pred = model.predict_proba(aligned_eval)
pred = [p[1] for p in pred] # Positive class (1) probabilities
pred[0]

# Safe to give this guy loan; < 50% is safe (0), > 50% is 1 (will default)

0.3067828